In [36]:
!pwd
!ls

/d/GH/GitWorkSpace/bank_model_competiton/data/202508251800
202508251800_gh_v1.csv
model_stacking.ipynb
process.ipynb
test.dat
test.dat.202508251800
train.dat
train.dat.202508251800


In [29]:
import pandas as pd 
import matplotlib.pyplot as plt
import statistics
import datetime
import seaborn as sns
import os
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

suffix = os.path.split(os.getcwd())[-1]

root_dir='../../'

train_path=root_dir+'train.csv'
train_tx_path=root_dir+'train_bank_statement.csv'
test_path=root_dir+'testaa.csv'
test_tx_path=root_dir+'testaa_bank_statement.csv'

In [50]:
print(suffix)
mode="test"
if mode == "train":
    base_path = train_path
    tx_path = train_tx_path
    output_path = "train.dat.%s" % suffix
else:
    base_path = test_path
    tx_path = test_tx_path
    output_path = "test.dat.%s" % suffix

print(base_path, tx_path, output_path)

202508251800
../../testaa.csv ../../testaa_bank_statement.csv test.dat.202508251800


In [51]:
tx_stat_d = dict()
df_bank = pd.read_csv(tx_path)
df_bank['time_format'] = df_bank['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
tx_dict = {key: group for key, group in df_bank.groupby('id')}
num = 0
for k,v in tx_dict.items():
    l = v.values
    tm_max = max(map(lambda x : x[4], l))
    tm_min = min(map(lambda x : x[4], l))
    tx_1 = filter(lambda x : x[2] == 1, l)
    tx_0 = filter( lambda x : x[2] == 0, l)
    total_amount = sum(list(map(lambda x : x[3], l)))
    amount_1 = sum(list(map(lambda x : x[3], tx_1)))
    amount_0 = sum(list(map(lambda x : x[3], tx_0)))
    tm_count = len(l)
    days_diff = (tm_max - tm_min).days
    total_amount_day_avg = round(total_amount / (days_diff+1), 4)
    amount_1_day_avg = round(amount_1 / (days_diff+1), 4)
    amount_0_day_avg = round(amount_0 / (days_diff+1), 4)
    output = [days_diff, tm_count, total_amount, amount_1, amount_0,total_amount_day_avg,amount_1_day_avg, amount_0_day_avg]
    tx_stat_d[k] = output


In [52]:
df_tx_stat = pd.DataFrame(tx_stat_d).T
df_tx_stat

,0,1,2,3,4,5,6,7
53480,165.0,46.0,61815.520000,11245.520000,50570.000000,372.3827,67.7441,304.6386
53481,144.0,56.0,38555.840000,20161.940000,18393.900000,265.9023,139.0479,126.8545
53482,172.0,74.0,81589.800000,44977.460000,36612.340000,471.6173,259.9853,211.6320
53483,159.0,73.0,48205.320000,24048.520000,24156.800000,301.2833,150.3032,150.9800
53484,174.0,91.0,78113.610000,29797.860000,48315.750000,446.3635,170.2735,276.0900
...,...,...,...,...,...,...,...,...
73515,94.0,25.0,9477.597620,5109.004148,4368.593472,99.7642,53.7790,45.9852
73517,180.0,272.0,212302.562407,126112.342194,86190.220212,1172.9423,696.7533,476.1891
73522,180.0,116.0,45701.276358,24957.995111,20743.281247,252.4932,137.8895,114.6038
73525,180.0,226.0,145276.809894,92246.439896,53030.369998,802.6343,509.6488,292.9855


In [53]:
df_tx_stat.shape

(8180, 8)

In [54]:
!pwd
df = pd.read_csv(base_path, index_col=['id'])
# print(df.head())
print(df.shape)
print(df_tx_stat.shape)
df_concat = pd.concat([df, df_tx_stat], axis=1)
print(df_concat.shape)


/d/GH/GitWorkSpace/bank_model_competiton/data/202508251800
(20054, 17)
(8180, 8)
(20054, 25)


In [56]:
df_concat.reset_index(inplace=True)
df_concat.rename(columns={'index':'id'}, inplace=True)
print(df_concat)
df_concat.to_csv(output_path, index=False)
print(output_path)
!ls

          id  title  career  zip_code  residence   loan  term  interest_rate  \
0      53480     10     5.0    512068          0   5950    36          14.10   
1      53481      8    10.0    401852          2  15350    36           5.25   
2      53482      8    10.0    321693          0  11050    36          13.65   
3      53483      8     7.0    231397          0  15350    36          14.20   
4      53484      7     7.0    171193          0   7800    36          10.00   
...      ...    ...     ...       ...        ...    ...   ...            ...   
20049  73529      0     8.0    601107          1  10000    12          18.85   
20050  73530      0    10.0    601102          1  10000    12          29.30   
20051  73531      0     4.0    601408          1  11000    12          24.75   
20052  73532      0     3.0    601904          1   8000    12          22.00   
20053  73533      2     1.0    601809          1   7000    12          14.95   

       issue_time  syndicated  ...   ba

In [49]:

df_concat


,id,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,...,level,label,0,1,2,3,4,5,6,7
0,0,9,0.0,221373,1,7200,36,10.95,1238631967,0,...,A4,0,163.0,48.0,71787.000000,12079.500000,59707.500000,437.7256,73.6555,364.0701
1,1,8,10.0,311681,0,21300,36,12.95,1128212052,0,...,B0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8,7.0,271562,1,10400,60,21.05,1249171509,0,...,B4,0,180.0,48.0,22406.100000,15883.720000,6522.380000,123.7906,87.7554,36.0352
3,3,7,2.0,522083,0,33050,36,16.40,1172882234,0,...,B3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,8,3.0,101026,1,5200,36,14.35,1172882384,0,...,B2,1,169.0,93.0,51163.000000,30823.100000,20339.900000,300.9588,181.3124,119.6465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53475,53475,2,2.0,603000,1,9000,12,23.55,1172880000,0,...,A4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53476,53476,0,10.0,601702,1,8000,12,30.70,1160092800,0,...,B2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53477,53477,2,10.0,602808,1,10000,12,9.40,1180310400,0,...,B2,0,180.0,251.0,113461.244335,69300.836223,44160.408112,626.8577,382.8775,243.9802
53478,53478,0,10.0,602102,2,9000,12,24.40,1176768000,0,...,A2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Help on method to_csv in module pandas.core.generic:

to_csv(path_or_buf: 'FilePathOrBuffer[AnyStr] | None' = None, sep: 'str' = ',', na_rep: 'str' = '', float_format: 'str | None' = None, columns: 'Sequence[Hashable] | None' = None, header: 'bool_t | list[str]' = True, index: 'bool_t' = True, index_label: 'IndexLabel | None' = None, mode: 'str' = 'w', encoding: 'str | None' = None, compression: 'CompressionOptions' = 'infer', quoting: 'int | None' = None, quotechar: 'str' = '"', line_terminator: 'str | None' = None, chunksize: 'int | None' = None, date_format: 'str | None' = None, doublequote: 'bool_t' = True, escapechar: 'str | None' = None, decimal: 'str' = '.', errors: 'str' = 'strict', storage_options: 'StorageOptions' = None) -> 'str | None' method of pandas.core.frame.DataFrame instance
    Write object to a comma-separated values (csv) file.
    
    Parameters
    ----------
    path_or_buf : str or file handle, default None
        File path or object, if None is provided the

## 交易数据

In [11]:
df_tx = pd.read_csv(train_tx_path)
# print(df_tx)
df_tx['time_format'] = df_tx['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
df_tx_stat = df_tx.groupby('id')['time_format'].agg(['max','min','count'])
df_tx_stat

,max,min,count
id,,,
0,2009-03-28 08:00:00,2008-10-16 08:00:00,48
2,2013-11-09 08:00:00,2013-05-13 08:00:00,48
4,2009-03-29 08:00:00,2008-10-11 08:00:00,93
6,2008-09-01 08:00:00,2008-03-06 08:00:00,61
7,2008-02-03 08:00:00,2007-08-12 08:00:00,66
...,...,...,...
53465,2006-10-22 08:00:00,2006-05-08 08:00:00,12
53467,2006-10-22 08:00:00,2006-04-25 08:00:00,281
53468,2006-10-22 08:00:00,2006-04-25 08:00:00,333


['a', 'b']
